In [2]:
import threading
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime, timedelta, timezone
from plotly.subplots import make_subplots
import talib
from talib import CDLENGULFING
import colored
import os
import yfinance
import time
from mpl_finance import candlestick_ohlc
import matplotlib.dates as mpl_dates
import matplotlib.pyplot as plt
if not os.path.exists("images"):
    os.mkdir("images")
from talib import EMA, MACD
from termcolor import colored
import chart_studio.plotly as py
from IPython.display import clear_output
import pyautogui
from scipy.stats import linregress
import pandas as pd
import yfinance as yf
import numpy as np
import math
from mplfinance.original_flavor import candlestick_ohlc
import matplotlib.dates as mdates
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, plot, iplot
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from scipy.signal import argrelextrema

In [3]:
import pytz
import MetaTrader5 as mt5
mt5.initialize()

True

In [4]:
def format_text(trend):
    if trend == 'Buy':
        return colored(trend, 'green')
    if trend == 'Sell':
        return colored(trend, 'red')
    return colored('Neutral', 'blue')

def heikin_ashi(df):
    heikin_ashi_df = pd.DataFrame(index=df.index.values, columns=['open', 'high', 'low', 'close'])
    
    heikin_ashi_df['close'] = (df['open'] + df['high'] + df['low'] + df['close']) / 4
    
    for i in range(len(df)):
        if i == 0:
            heikin_ashi_df.iat[0, 0] = df['open'].iloc[0]
        else:
            heikin_ashi_df.iat[i, 0] = (heikin_ashi_df.iat[i-1, 0] + heikin_ashi_df.iat[i-1, 3]) / 2
        
    heikin_ashi_df['high'] = heikin_ashi_df.loc[:, ['open', 'close']].join(df['high']).max(axis=1)
    
    heikin_ashi_df['low'] = heikin_ashi_df.loc[:, ['open', 'close']].join(df['low']).min(axis=1)
    heikin_ashi_df['Date'] = df['Date']
    return heikin_ashi_df

def format_text(trend):
    if trend == 'Buy':
        return colored(trend, 'green')
    if trend == 'Sell':
        return colored(trend, 'red')
    if trend == 'Close_Buy':
        return colored(trend, 'yellow')
    if trend == 'Close_Sell':
        return colored(trend, 'yellow')
    if trend == 'Neutral':
        return colored(trend, 'cyan')
    return colored('0', 'blue')
def calculate_slope(signal):
    x = np.array([x for x in range(len(signal.index))])
    scaler = MinMaxScaler(feature_range=(0, 100))
    y = scaler.fit_transform(signal.values.reshape(-1, 1))
    slope, intercept, r_value, p_value, std_err = linregress(x, y.flatten())
    return slope

In [5]:
def get_trend(close_p: float, sp_trend_1: float) -> str:
    if close_p > sp_trend_1:
        return "Buy"
    if close_p < sp_trend_1:
        return "Sell"
    return "No Trend"

def isSupport(df,i):
    support = df['Low'][i] < df['Low'][i-1] and \
        df['Low'][i] < df['Low'][i+1] and \
        df['Low'][i+1] < df['Low'][i+2] and \
        df['Low'][i-1] < df['Low'][i-2]

    return support


def isResistance(df,i):
    resistance = df['High'][i] > df['High'][i-1]  and \
        df['High'][i] > df['High'][i+1] and \
        df['High'][i+1] > df['High'][i+2] and \
        df['High'][i-1] > df['High'][i-2] 

    return resistance


def isFarFromLevel(l, s, levels):
    return np.sum([abs(l-x[1]) < s  for x in levels]) == 0


def calculate_support_registance(start, stop):
    levels = []
    df_new = df.iloc[start:stop].reset_index(drop=True)
    s =  np.mean(df_new['High'] - df_new['Low'])
    for i in range(2,df_new.shape[0]-2):
        if isSupport(df_new,i):
            l = df_new['Low'][i]
            if isFarFromLevel(l, s, levels):
                levels.append((i,l, 'Support'))

        elif isResistance(df_new,i):
            l = df_new['High'][i]
            if isFarFromLevel(l, s, levels):
                levels.append((i,l, 'Resistance'))
    return levels

def save_result(order_index, df_index, trend):
    return True
    new_df = df.iloc[order_index:df_index]
    data=[
        go.Candlestick(x=new_df.index, open=new_df['open'], high=new_df['high'], low=new_df['low'], close=new_df['close']),
    ]
    fig = go.Figure(data)
    fig.add_hline(y=df.close[order_index])
#     fig.add_vline(x=df.index[order_index], line_width=3, line_dash="dash", line_color="green")
    fig.update_layout(xaxis_rangeslider_visible=False)
    fig.write_image(f"images/{order_index}-{trend}.png")
    
def finonaci(df_check):
    price_min = df_check.Close.min()
    price_max = df_check.Close.max()
    diff = abs(price_max - price_min)
    level1 = 0.236 * diff
    level2 = price_max - 0.382 * diff
    level3 = price_max - 0.618 * diff
#     print(price_min, price_max, diff, level1)
    return level1

In [6]:
def get_valid_uploaded_filename(filename, folder):
        """
        Check whether uploaded file already exists. If yes, rename it

        :param filename: uploaded file
        :type filename: str
        :param folder: folder into save the file
        :type folder: str
        :return: unique filename
        :rtype: str
        """
        file_query_s = os.path.join(folder, filename)
        i = 2
        filename_orig = filename
        while os.path.exists(file_query_s):
            filename = str(i) + "_" + filename_orig
            file_query_s = os.path.join(folder, filename)
            i += 1
        return filename 
    
def plot_chart(df, pivots, order_label, order_date, stop_loss, signal_slope, x_signal, minima, maxima):
    # Set colours for up and down candles
    INCREASING_COLOR = '#26a69a'
    DECREASING_COLOR = '#ef5350'
    
    # create list to hold dictionary with data for our first series to plot
    # (which is the candlestick element itself)
    data = [ 
        dict(
            type = 'candlestick',
            open = df.open,
            high = df.high,
            low = df.low,
            close = df.close,
            x = df.Date,
            yaxis = 'y3',
            name = 'candle stick chart',
            increasing = dict( line = dict( color = INCREASING_COLOR ) ),
            decreasing = dict( line = dict( color = DECREASING_COLOR ) ),
        ),dict(
            type = 'scatter',
            name = "SIGNAL", 
            mode = "lines",
            x = df.Date,
            y = df.SIGNAL,
            yaxis = 'y2'
        ),dict(
            type = 'scatter',
            name = "MACD", 
            x = df.Date,
            y = df.MACD,
            yaxis = 'y2'
        ),dict(
            type = 'bar',
            name = "MACD Histogram", 
            x = df.Date,
            y = df.HIST,
            yaxis = 'y2'
        ),dict(
            type = 'scatter',
            mode = "lines", 
            name = "RSI", 
            x = df.Date,
            y = df.RSI
        )
    ]

    # Create empty dictionary for later use to hold settings and layout options
    layout = {
      "yaxis": {"domain": [0, 0.33], "visible": False}, 
      "yaxis2": {"domain": [0.33, 0.66], "visible": False}, 
      "yaxis3": {"domain": [0.66, 1], "visible": False},
      "xaxis": {"rangeslider": {"visible": False}, "visible": False},
      'showlegend': False
    }

    # create our main chart "Figure" object which consists of data to plot and layout settings
    fig = dict(data=data, layout=layout)

    # Assign various seeting and choices - background colour, range selector etc
    fig['layout']['plot_bgcolor'] = 'rgb(250, 250, 250)'
    #fig['layout']['xaxis'] = dict( rangeselector = dict( visible = False ) )
#     fig['layout']['yaxis'] = dict( domain = [0, 0.2], showticklabels = False )
#     fig['layout']['yaxis2'] = dict( domain = [0.2, 0.8] )
    #fig['layout']['legend'] = dict( orientation = 'h', y=0.9, x=0.3, yanchor='bottom' )
    fig['layout']['margin'] = dict( t=40, b=40, r=40, l=40 )
    fig['layout']['width'] = 1920
    fig['layout']['height'] = 1080
    
    # Populate the "rangeselector" object with necessary settings
    rangeselector=dict(
        visible = False,
        x = 0, y = 0.9,
        bgcolor = 'rgba(150, 200, 250, 0.4)',
        font = dict( size = 13 ),
        buttons=list([
            dict(count=1,
                 label='reset',
                 step='all'),
            dict(count=1,
                 label='1yr',
                 step='year',
                 stepmode='backward'),
            dict(count=3,
                label='3 mo',
                step='month',
                stepmode='backward'),
            dict(count=1,
                label='1 mo',
                step='month',
                stepmode='backward'),
            dict(step='all')
        ]))


    # fig['layout']['xaxis']['rangeselector'] = rangeselector

    # Append the Ichimoku elements to the plot
    fig['data'].append( dict( x=df.Date, y=df['tenkan_sen'], type='scatter', mode='lines', 
                             line = dict( width = 2 ),
                             marker = dict( color = '#33BDFF' ),
                             yaxis = 'y3', name='tenkan_sen' ) )

    fig['data'].append( dict( x=df.Date, y=df['kijun_sen'], type='scatter', mode='lines', 
                             line = dict( width = 2 ),
                             marker = dict( color = '#F1F316' ),
                             yaxis = 'y3', name='kijun_sen' ) )

    fig['data'].append( dict( x=df.Date, y=df['senkou_span_a'], type='scatter', mode='lines', 
                             line = dict( width = 1 ), 
                             marker = dict( color = '#228B22' ),
                             yaxis = 'y3', name='senkou_span_a' ) )

    fig['data'].append( dict( x=df.Date, y=df['senkou_span_b'], type='scatter', mode='lines', 
                             line = dict( width = 1 ),fill='tonexty',
                             marker = dict( color = '#FF3342' ),
                             yaxis = 'y3', name='senkou_span_b' ) )

    fig['data'].append( dict( x=df.Date, y=df['chikou_span'], type='scatter', mode='lines', 
                             line = dict( width = 1 ),
                             marker = dict( color = '#D105F5' ),
                             yaxis = 'y3', name='chikou_span' ) )
    have_resistance = False
    have_support = False
    for pivot in reversed(pivots):
        pivot_idx, pivot_price, pivot_type = pivot
        if pivot_type == "resistance":
            have_resistance = True
        if pivot_type == "support":
            have_support = True

        y_sample = [pivot_price if pivot_idx < index else None for index in range(len(df.index))]
        x_sample = [date if pivot_idx < index else None for index, date in enumerate(df['Date'])]

        fig['data'].append( dict( x=x_sample, y=y_sample, type='scatter', mode='lines', 
                             line = dict( width = 3 ),
                             marker = dict( color = '#eb346e' if pivot_type == "resistance" else "#52eb34" ),
                             yaxis = 'y3', name='pivots' ) )
        if have_support and have_resistance:
            break
    if stop_loss:
        y_sample = [stop_loss]*len(df.index)
        fig['data'].append( dict( x=df['Date'], y=y_sample, type='scatter', mode='lines', 
                     line = dict( width = 3 ),
                     marker = dict( color = '#eb346e' if pivot_type == "resistance" else "#52eb34" ),
                     yaxis = 'y3', name='pivots' ) )

    # Set colour list for candlesticks
    colors = []

    for i in range(len(df.close)):
        if i != 0:
            if df.close[i] > df.close[i-1]:
                colors.append(INCREASING_COLOR)
            else:
                colors.append(DECREASING_COLOR)
        else:
            colors.append(DECREASING_COLOR)
    for index, date in enumerate(df['Date']):
        if date == order_date:
            # print(order_date, df.close.iat[index])
            fig['data'].append(dict(x=[order_date], y=[df.close.iat[index]], yaxis = 'y3', marker=dict(color='#3256a8')))
            break
    
    for item in minima:
        fig['data'].append(dict(x=[df['Date'].iat[item]], y=[df.SIGNAL.iat[item]], yaxis = 'y2', marker=dict(color='#42f554')))

    for item in maxima:
        fig['data'].append(dict(x=[df['Date'].iat[item]], y=[df.SIGNAL.iat[item]], yaxis = 'y2', marker=dict(color='#f54254')))

    figg = go.Figure(fig)
    order_date = order_date.replace(":", "")
    if order_label >= 0:
        file_name = f"images/1/{order_date} {order_label} {signal_slope}.png"
    else:
        file_name = f"images/0/{order_date} {order_label} {signal_slope}.png"
    figg.write_image(file_name)
    # py.iplot(fig, filename='Sine Wave Slider')

In [7]:
import matplotlib.pyplot as plt
#method 2: window shifting method
#using the same symbol as the first example above

# for visualization
def plot_all(levels, df):    
#     fig, ax = plt.subplots(figsize=(16, 9))   
#     date_format = mpl_dates.DateFormatter('%Y-%b-%d')
    f1 = plt.subplot2grid((6, 4), (1, 0), rowspan=6, colspan=4)
    # f1.xaxis.set_major_formatter(mdates.DateFormatter('%y-%m-%d'))
    candlestick_ohlc(f1, df.values, width=.6, colorup='#53c156', colordown='#ff1717')
    for level in levels:        
        plt.hlines(level[1], xmin = df['index'][level[0]], xmax = max(df['index']), colors='blue', linestyle='--')    
    plt.show()

def is_support(df,i):  
    cond1 = df['low'][i] < df['low'][i-1]   
    cond2 = df['low'][i] < df['low'][i+1]   
    cond3 = df['low'][i+1] < df['low'][i+2]   
    cond4 = df['low'][i-1] < df['low'][i-2]  
    return (cond1 and cond2 and cond3 and cond4) 
# determine bearish fractal
def is_resistance(df,i):  
    cond1 = df['high'][i] > df['high'][i-1]   
    cond2 = df['high'][i] > df['high'][i+1]   
    cond3 = df['high'][i+1] > df['high'][i+2]   
    cond4 = df['high'][i-1] > df['high'][i-2]  
    return (cond1 and cond2 and cond3 and cond4)

# to make sure the new level area does not exist already
def is_far_from_level(value, levels, df):    
    ave =  np.mean(df['high'] - df['low'])    
    return np.sum([abs(value-level)<ave for _,level,_ in levels])==0

def get_pivot(df):
    pivots = []
    max_list = []
    min_list = []
    for i in range(5, len(df)-5):
        # taking a window of 9 candles
        high_range = df['close'][i-5:i+4]
        current_max = high_range.max()
        # if we find a new maximum value, empty the max_list 
        if current_max not in max_list:
            max_list = []
        max_list.append(current_max)
        # if the maximum value remains the same after shifting 5 times
        if len(max_list)==5 and is_far_from_level(current_max,pivots,df):
            pivots.append((high_range.idxmax(), current_max, "resistance"))

        low_range = df['close'][i-5:i+5]
        current_min = low_range.min()
        if current_min not in min_list:
            min_list = []
        min_list.append(current_min)
        if len(min_list)==5 and is_far_from_level(current_min,pivots,df):
            pivots.append((low_range.idxmin(), current_min, "support"))
    return pivots


# test_ragnge = 1000
# for idx in range(test_ragnge):
# mt5.copy_rates_from()
# rates_frame = mt5.copy_rates_from('BTCUSDm', mt5.TIMEFRAME_H1, datetime.now(), 150)
# df = pd.DataFrame(rates_frame, columns=['time', 'open', 'high', 'low', 'close'])
# df['Date'] = pd.to_datetime(df['time'], unit='s')
# df = df.drop(['time'], axis=1)
# df['Date'] = df.Date.dt.strftime('%Y.%m.%d %H:%M:%S')
# df = df.reindex(columns=['Date', 'open', 'high', 'low', 'close'])
# # convert to float to avoid sai so.
# df.open = df.open.astype(float)
# df.high = df.high.astype(float)
# df.low = df.low.astype(float)
# df.close = df.close.astype(float)
# df['RSI'] = talib.RSI(df.close)
# exp1 = df.close.ewm(span=12, adjust=False).mean()
# exp2 = df.close.ewm(span=26, adjust=False).mean()
# df['MACD'] = exp1 - exp2
# df['SIGNAL'] = df['MACD'].ewm(span=9, adjust=False).mean()
# df['HIST'] = df['MACD'] - df['SIGNAL']
# df['HIST_NORM'] = df.HIST.ewm(span=20, adjust=False).mean()
# # Tenkan-sen (Conversion Line): (9-period high + 9-period low)/2))
# nine_period_high = df['high'].rolling(window= 9).max()
# nine_period_low = df['low'].rolling(window= 9).min()
# df['tenkan_sen'] = (nine_period_high + nine_period_low) /2
# # Kijun-sen (Base Line): (26-period high + 26-period low)/2))
# period26_high = df['high'].rolling(window=26).max()
# period26_low = df['low'].rolling(window=26).min()
# df['kijun_sen'] = (period26_high + period26_low) / 2
# # Senkou Span A (Leading Span A): (Conversion Line + Base Line)/2))
# df['senkou_span_a'] = ((df['tenkan_sen'] + df['kijun_sen']) / 2).shift(26)
# # Senkou Span B (Leading Span B): (52-period high + 52-period low)/2))
# period52_high = df['high'].rolling(window=52).max()
# period52_low = df['low'].rolling(window=52).min()
# df['senkou_span_b'] = ((period52_high + period52_low) / 2).shift(26)
# # The most current closing price plotted 26 time periods behind (optional)
# df['chikou_span'] = df['close'].shift(-26)
# # Tenkan-sen (Conversion Line): (9-period high + 9-period low)/2))
# macd, macdsignal, macdhist = talib.MACD(df.close)
# macd = macdsignal.dropna()[-15:]
# pivots = get_pivot(df)
# plot_chart(df, pivots, "", "", None)
#time.sleep(0.25)
#clear_output(wait=False)


In [20]:
def ichimoku_cloud(utc_to, timeframe, symbol, save_frame, order_label, order_date, stop_loss):
    rates_frame = mt5.copy_rates_from(symbol, timeframe, utc_to + timedelta(hours=2), 100)
    rates_frame = rates_frame.copy()
    # create DataFrame out of the obtained data
    
    if timeframe == mt5.TIMEFRAME_W1:
        num_frame = utc_to.weekday() + 1
        rates_frame_fixed = mt5.copy_rates_from(symbol, mt5.TIMEFRAME_D1, utc_to + timedelta(hours=2), num_frame)
        df_fixed = pd.DataFrame(rates_frame_fixed, columns=['time', 'open', 'high', 'low', 'close'])
        df_fixed['Date'] = pd.to_datetime(df_fixed['time'], unit='s')
        df_fixed['Date'] = df_fixed.Date.dt.strftime('%Y.%m.%d %H:%M:%S')
        df_fixed = df_fixed.drop(['time'], axis=1)
        df_fixed = df_fixed.reindex(columns=['Date', 'open', 'high', 'low', 'close'])

        new_open, new_high, new_low, new_close = df_fixed.open.iat[0], np.amax(df_fixed.high), np.amin(df_fixed.low), df_fixed.close.iat[-1]
        current_date = pd.to_datetime(rates_frame[-1][0], unit='s').strftime('%Y.%m.%d %H:%M:%S')

        rates_frame[-1][0] = rates_frame_fixed[-1][0]
        rates_frame[-1][1] = new_open
        rates_frame[-1][2] = new_high
        rates_frame[-1][3] = new_low
        rates_frame[-1][4] = new_close

    if timeframe == mt5.TIMEFRAME_D1:
        num_frame = utc_to.hour % 24 + 1
        rates_frame_fixed = mt5.copy_rates_from(symbol, mt5.TIMEFRAME_H1, utc_to + timedelta(hours=2), num_frame)
        df_fixed = pd.DataFrame(rates_frame_fixed, columns=['time', 'open', 'high', 'low', 'close'])
        df_fixed['Date'] = pd.to_datetime(df_fixed['time'], unit='s')
        df_fixed['Date'] = df_fixed.Date.dt.strftime('%Y.%m.%d %H:%M:%S')
        df_fixed = df_fixed.drop(['time'], axis=1)
        df_fixed = df_fixed.reindex(columns=['Date', 'open', 'high', 'low', 'close'])
 
        new_open, new_high, new_low, new_close = df_fixed.open.iat[0], np.amax(df_fixed.high), np.amin(df_fixed.low), df_fixed.close.iat[-1]
        current_date = pd.to_datetime(rates_frame[-1][0], unit='s').strftime('%Y.%m.%d %H:%M:%S')

        rates_frame[-1][0] = rates_frame_fixed[-1][0]
        rates_frame[-1][1] = new_open
        rates_frame[-1][2] = new_high
        rates_frame[-1][3] = new_low
        rates_frame[-1][4] = new_close

    if timeframe == mt5.TIMEFRAME_H12:
        num_frame = utc_to.hour % 12 + 1
        rates_frame_fixed = mt5.copy_rates_from(symbol, mt5.TIMEFRAME_H1, utc_to + timedelta(hours=2), num_frame)
        df_fixed = pd.DataFrame(rates_frame_fixed, columns=['time', 'open', 'high', 'low', 'close'])
        df_fixed['Date'] = pd.to_datetime(df_fixed['time'], unit='s')
        df_fixed['Date'] = df_fixed.Date.dt.strftime('%Y.%m.%d %H:%M:%S')
        df_fixed = df_fixed.drop(['time'], axis=1)
        df_fixed = df_fixed.reindex(columns=['Date', 'open', 'high', 'low', 'close'])
        
        new_open, new_high, new_low, new_close = df_fixed.open.iat[0], np.amax(df_fixed.high), np.amin(df_fixed.low), df_fixed.close.iat[-1]
        current_date = pd.to_datetime(rates_frame[-1][0], unit='s').strftime('%Y.%m.%d %H:%M:%S')

        rates_frame[-1][0] = rates_frame_fixed[-1][0]
        rates_frame[-1][1] = new_open
        rates_frame[-1][2] = new_high
        rates_frame[-1][3] = new_low
        rates_frame[-1][4] = new_close

    df = pd.DataFrame(rates_frame, columns=['time', 'open', 'high', 'low', 'close'])
    df['Date'] = pd.to_datetime(df['time'], unit='s')
    df['Date'] = df.Date.dt.strftime('%Y.%m.%d %H:%M:%S')
    df = df.drop(['time'], axis=1)
    df = df.reindex(columns=['Date', 'open', 'high', 'low', 'close'])
    # convert to float to avoid sai so.
    df.open = df.open.astype(float)
    df.high = df.high.astype(float)
    df.low = df.low.astype(float)
    df.close = df.close.astype(float)
    real_close = df.close.iat[-1]
    real_high = df.high.iat[-1]
    real_low = df.low.iat[-1]
    df['ATR'] = talib.ATR(df.high, df.low, df.close)
    df['RSI'] = talib.RSI(df.close)
    df['MACD'], df['SIGNAL'], df['HIST'] = MACD(df.close)
    nine_period_high = df['high'].rolling(window= 9).max()
    nine_period_low = df['low'].rolling(window= 9).min()
    df['tenkan_sen'] = (nine_period_high + nine_period_low) /2
    # Kijun-sen (Base Line): (26-period high + 26-period low)/2))
    period26_high = df['high'].rolling(window=26).max()
    period26_low = df['low'].rolling(window=26).min()
    df['kijun_sen'] = (period26_high + period26_low) / 2
    # Senkou Span A (Leading Span A): (Conversion Line + Base Line)/2))
    df['senkou_span_a'] = ((df['tenkan_sen'] + df['kijun_sen']) / 2).shift(26)
    # Senkou Span B (Leading Span B): (52-period high + 52-period low)/2))
    period52_high = df['high'].rolling(window=52).max()
    period52_low = df['low'].rolling(window=52).min()
    df['senkou_span_b'] = ((period52_high + period52_low) / 2).shift(26)
    # The most current closing price plotted 26 time periods behind (optional)
    df['chikou_span'] = df['close'].shift(-26)
    # create a quick plot of the results to see what we have created
        
    x_signal = df.SIGNAL.values
    
    maxima = argrelextrema(np.array(x_signal), np.greater)[0]
    maxima_val = [x_signal[max_val] for max_val in maxima]
    minima = argrelextrema(np.array(x_signal), np.less)[0]
    minima_val = [x_signal[min_val] for min_val in minima]

    # calculate slope
    tenkan_array = df.tenkan_sen[-5:]
    close_array = df.close[-5:]
    kinjun_array = df.kijun_sen[-5:]
    macd_array = df.MACD[-5:]
    signal_array = df.SIGNAL[-5:]
    hist_array = df.HIST[-5:]

    tenkan_slope = calculate_slope(tenkan_array)
    close_slope = calculate_slope(close_array)
    kinjun_slope = calculate_slope(kinjun_array)
    macd_slope = calculate_slope(macd_array)
    signal_slope = calculate_slope(signal_array)
    hist_slope = calculate_slope(hist_array)
    
    input_df_1 = df['tenkan_sen'][-14:]
    input_df_2 = df['kijun_sen'][-14:]
    input_df_3 = df['senkou_span_a'][-14:]
    input_df_4 = df['senkou_span_b'][-14:]
    input_df_5 = df['HIST'][-14:]
    input_df_6 = df['MACD'][-14:]
    input_df_7 = df['SIGNAL'][-14:]
    input_df_8 = df['close'][-14:]
    input_df_9 = df['RSI'][-14:]
    input_df_10 = df['ATR'][-14:]
#     if df.Buy_Signal.iat[-1] != "0":
#         print(signal_slope, price_slope, hist_slope)

    normal_array = [calculate_slope(x) for x in [input_df_1, input_df_2, input_df_3, input_df_4, input_df_5, input_df_6, input_df_7, input_df_8, input_df_9, input_df_10]]

    slope = 0
    if hist_slope > 0 and macd_slope > 0 and close_slope > 0:
#         if (len(minima_val) > 0 and minima_val[-1] < 0) or (len(minima_val) >= 2 and df.SIGNAL.iat[-1] > minima_val[-2]):
            # detect minimal of signal
        slope = 1

    elif hist_slope < 0 and macd_slope < 0 and close_slope < 0:
#         if (len(maxima_val) > 0 and maxima_val[-1] > 0) or (len(maxima_val) >= 2 and df.SIGNAL.iat[-1] < maxima_val[-2]):
            # detect minimal of signal
        slope = -1

#     if timeframe != mt5.TIMEFRAME_H4:
#         conditions = [
#             (df['kijun_sen'] < df['close']) & (df['HIST'] > df['HIST'].shift()),
#             (df['kijun_sen'] > df['close']) & (df['HIST'] < df['HIST'].shift())
#         ]
#     else:
    conditions = [
        (df['tenkan_sen'] > df['senkou_span_a']) & (df['tenkan_sen'] > df['senkou_span_b']) &
        (df['kijun_sen'] < df['close']) & (df['HIST'] > df['HIST'].shift()),
        (df['tenkan_sen'] < df['senkou_span_a']) & (df['tenkan_sen'] < df['senkou_span_b']) &
        (df['kijun_sen'] > df['close']) & (df['HIST'] < df['HIST'].shift())
    ]

    values = ['Buy', 'Sell']
    df['Buy_Signal'] = np.select(conditions, values)

    conditions = [
        (df['kijun_sen'] > df['close']),
        (df['kijun_sen'] < df['close'])
    ]
    values = ['Close_Buy', 'Close_Sell']
    df['Close_Signal'] = np.select(conditions, values)

    mapping_timeframe = {
        mt5.TIMEFRAME_H1: "H1",
        mt5.TIMEFRAME_M15: "M15",
        mt5.TIMEFRAME_M5: "M5",
        mt5.TIMEFRAME_M30: "M30",
        mt5.TIMEFRAME_D1: "D1",
        mt5.TIMEFRAME_H4: "H4",
        mt5.TIMEFRAME_H12: "H12",
    }
    map_timeframe = mapping_timeframe.get(timeframe)
    
    pivots = get_pivot(df)
    resistance, support = None, None
    if len(pivots) >= 2:
        pivot_1 = pivots[-1][1]
        pivot_2 = pivots[-2][1]
        if pivot_1 > pivot_2:
            resistance, support = pivot_1, pivot_2
        else:
            resistance, support = pivot_2, pivot_1
    
    if save_frame:
        date_save = str(df.Date.iat[-1]).replace(":", "")
        plot_chart(df, pivots, order_label, order_date, stop_loss, signal_slope, x_signal, minima, maxima)

    return str(df.Date.iat[-1]), df.Buy_Signal.iat[-1], df.Close_Signal.iat[-1], real_close, real_high, real_low, df.kijun_sen.iat[-1], df['ATR'].iat[-1], normal_array, resistance, support, slope, 0, df

In [21]:
X = []
y = []


def thread_mt5(symbol):
    # reset flags
    global X
    global y
    order_p = None
    stop_loss = None
    order_size = None
    order_index = None
    profit = 0
    max_profit = 0
    bet = 0.01
    max_bet = 10
    max_loss = 0
    num_loss = 0
    num_profit = 0
    take_profit = None
    all_profit = []
    all_diff = []
    with_ema = True
    with_stoploss = True
    with_dx = True
    adx_threshold = 0
    cost = 0.1
    lock_back_frame = 14
    max_diff = 0
    min_diff = 0
    threshold = 0.95
    waiting_frame = 0
    accept_next = ""
    order_date = ""
    max_diff_date = ""
    prev_time = ""
    all_max_diff = []
    all_min_diff = []
    x_train_max = []
    all_slope = []
    timezone = pytz.timezone("Europe/Athens")
    timeframe_1 = mt5.TIMEFRAME_H1
    timeframe_2 = mt5.TIMEFRAME_H4
    timeframe_3 = mt5.TIMEFRAME_H12
    timeframe_4 = mt5.TIMEFRAME_M30
    time_inremental = timedelta(hours=1)
    current_time = datetime(2018, 10, 11, 0, 0, 0, tzinfo=timezone)
#     stop_time = datetime(2017, 10, 10, 0, 0, 0, tzinfo=timezone)
    
    # current_time = datetime.utcnow() - timedelta(days=30)
#     stop_time = datetime.utcnow() + timedelta(days=2)
#     current_time = datetime.now(tz=timezone) - timedelta(days=5)
    stop_time = datetime.now(tz=timezone) + timedelta(days=2)
    print(current_time)
    print(stop_time)
    while current_time.timestamp() < stop_time.timestamp():
        # print(current_time.minute)
        df1date, h1_trend, close_signal_1, current_price_1, high_price_1, low_price_1, kijun_sen_1, atr_1, dftrain_1, resistance_1, support_1, slope_1, signal_slope_1, dataframe_1 = ichimoku_cloud(utc_to=current_time, timeframe=timeframe_1, symbol=symbol, save_frame=False, order_label="", order_date="", stop_loss=None)
        df2date, h2_trend, close_signal_2, current_price_2, high_price_2, low_price_2, kijun_sen_2, atr_2, dftrain_2, resistance_2, support_2, slope_2, signal_slope_2, dataframe_2 = ichimoku_cloud(utc_to=current_time, timeframe=timeframe_2, symbol=symbol, save_frame=False, order_label="", order_date="", stop_loss=None)
        df3date, h3_trend, close_signal_3, current_price_3, high_price_3, low_price_3, kijun_sen_3, atr_3, dftrain_3, resistance_3, support_3, slope_3, signal_slope_3, dataframe_3 = ichimoku_cloud(utc_to=current_time, timeframe=timeframe_3, symbol=symbol, save_frame=False, order_label="", order_date="", stop_loss=None)
        #df4date, h4_trend, close_signal_4, current_price_4, high_price_4, low_price_4, kijun_sen_4, atr_4, dftrain_4, resistance_4, support_4, slope_4, signal_slope_4, dataframe_4 = ichimoku_cloud(utc_to=current_time, timeframe=timeframe_4, symbol=symbol, save_frame=False, order_label="", order_date="", stop_loss=None)

        if df1date == prev_time:
            current_time += time_inremental
            continue

        current_price = current_price_1
        current_trend = "0"
        if h1_trend == h2_trend == h3_trend == "Sell":
            current_trend = "Sell"
        elif h1_trend == h2_trend == h3_trend == "Buy":
            current_trend = "Buy"
        elif close_signal_1 == "Close_Sell":
            current_trend = "Close_Sell"
        elif close_signal_1 == "Close_Buy":
            current_trend = "Close_Buy"
            
        if order_p and order_size == "Sell" and (current_trend == "Neutral" or current_trend == 'Buy' or h1_trend == "Buy" or h2_trend == "Buy" or current_trend == 'Close_Sell' or (stop_loss and stop_loss < high_price_1) or current_time.timestamp() > stop_time.timestamp()): 
            diff = order_p - current_price
            if stop_loss and stop_loss < high_price_1:
                diff = order_p - stop_loss

            print(f"{df1date} close \x1B[31m{order_size}\x1b[0m order at price {current_price}, max diff {round(max_diff, 5)} diff {round(diff, 5)}")
            ichimoku_cloud(utc_to=current_time, timeframe=timeframe_1, symbol=symbol, save_frame=True, order_label=diff, order_date=order_date, stop_loss=stop_loss)
            #ichimoku_cloud(utc_to=current_time, timeframe=timeframe_2, symbol=symbol, save_frame=True, order_label=f"Close Sell {round(diff, 5)}", order_date=order_date)
            profit += (diff)
            
            all_profit.append(profit)
#             all_diff.append(diff)
            if diff < 0:
                all_min_diff.append(diff)
                max_loss += diff
                num_loss += 1
                label = 0
            elif diff > 0:
                all_max_diff.append(max_diff)
                max_profit += diff
                num_profit += 1
                label = 1  
            # y.append(label)
            # reset flags
            order_p = None
            order_size = None
            stop_loss = None
            take_profit = None
            max_diff = 0
            if current_time.timestamp() > stop_time.timestamp():
                break
        elif order_p and order_size == "Buy" and (current_trend == "Neutral" or current_trend == 'Sell' or h1_trend == "Sell" or h2_trend == "Sell" or current_trend == 'Close_Buy' or (stop_loss and stop_loss > low_price_1) or current_time.timestamp() > stop_time.timestamp()):
            diff = current_price - order_p
            if stop_loss and stop_loss > low_price_1:
                diff = stop_loss - order_p
            ichimoku_cloud(utc_to=current_time, timeframe=timeframe_1, symbol=symbol, save_frame=True, order_label=diff, order_date=order_date, stop_loss=stop_loss)
            #ichimoku_cloud(utc_to=current_time, timeframe=timeframe_2, symbol=symbol, save_frame=True, order_label=f"Close Buy {round(diff, 5)}", order_date=order_date)
            print(f"{df1date} close \x1b[48;5;4m{order_size}\x1b[0m order at price {current_price}, max diff {round(max_diff, 5)} diff {round(diff, 5)}")
            profit += (diff)
            
            all_diff.append(diff)
            all_profit.append(profit)
            if diff < 0:
                all_min_diff.append(diff)
                max_loss += diff
                num_loss += 1
                label = 0
            elif diff > 0:
                all_max_diff.append(max_diff)
                max_profit += diff
                num_profit += 1
                label = 1

#             X.append(x_train_max)
#             label = pyautogui.confirm(text='Are you sure?', title='Confirm delete', buttons=["1", "0"])
#             y.append(int(label))

            # reset flags
            order_p = None
            order_size = None
            stop_loss = None
            take_profit = None
            max_diff = 0
            if current_time.timestamp() > stop_time.timestamp():
                break
        elif order_p is None and current_trend == "Sell" and slope_1 == -1 and support_1 and current_price < support_1:
#             all_slope.append(signal_slope_1)
            order_p = current_price
            order_size = current_trend
            order_date = df1date
            stop_loss = kijun_sen_1
            take_profit = None
            max_diff = 0
            accept_next = ""
            print(f"{df1date} new \x1B[31m{order_size}\x1b[0m order at price {current_price}")
#             ichimoku_cloud(utc_to=current_time, timeframe=timeframe_1, symbol=symbol, save_frame=True, order_label=f"Sell {predict_1}", order_date=df1date)

        elif order_p is None and current_trend == "Buy" and slope_1 == 1 and resistance_1 and current_price > resistance_1:
            all_slope.append(signal_slope_1)
            order_p = current_price
            order_size = current_trend
            order_date = df1date
            stop_loss = kijun_sen_1
            take_profit = None
            accept_next = ""
            max_diff = 0
            x_train_max = dataframe_1
            print(f"{df1date} new \x1b[48;5;2m{order_size}\x1b[0m order at price {current_price}")
            # ichimoku_cloud(utc_to=current_time, timeframe=timeframe_1, symbol=symbol, save_frame=True, order_label=f"Buy {predict_1}", order_date=df1date)
            # clear_output(wait=True)

        if order_size == "Sell":
            diff = order_p - current_price
            if diff > max_diff:
                max_diff = diff
            if stop_loss is None and diff > atr_1:
                stop_loss = order_p
            if stop_loss and stop_loss > kijun_sen_1 > current_price:
                stop_loss = kijun_sen_1
        elif order_size == "Buy":
            diff = current_price - order_p
            if diff > max_diff:
                max_diff = diff 
                max_diff_date = current_time
            if stop_loss is None and diff > atr_1:
                stop_loss = order_p
            if stop_loss and stop_loss < kijun_sen_1 < current_price:
                stop_loss = kijun_sen_1

        current_time += time_inremental
        prev_time = df1date

    print(f"symbol {symbol}")
    print(f"total trade: {len(all_profit)} avg loss: {round(max_loss/num_loss, 1)} / {num_loss} avg profit {round(max_profit/num_profit, 1)} / {num_profit}")
    print(f"total loss: {round(max_loss, 5)} total profit {round(max_profit, 5)} real profit {round(profit, 5)}")
    print(f"all max diff avg {sum(all_max_diff) / len(all_max_diff)}")
    print(f"all min diff avg {sum(all_min_diff) / len(all_min_diff)}")
    print(f"ROI { round((max_profit/num_profit)/(-max_loss/num_loss), 1) }")
    print(f"winrate {round(num_profit/len(all_profit)*100, 1)}% total trade {len(all_profit)}")
    # Initialize figure with subplots
    fig = make_subplots(
        rows=2, cols=1,
        column_widths=[2],
        row_heights=[1, 0.5])
    #     specs=[[{"type": "scatter", "colspan": 2}, {"type": "scatter"}],
    #            [            None                    , {"type": "bar"}]])
    # Add locations bar chart
    fig.add_trace(
        go.Bar(x=['total profit', 'total loss'],  y=[max_profit, abs(max_loss)], showlegend=False, 
               text=[f"avg profit {round(max_profit/num_profit, 2)} / {num_profit}", 
                     f"avg loss: {round(max_loss/num_loss, 2)} / {num_loss}"], textposition='auto'),
        row=2, col=1
    )
    fig.add_trace(
        go.Scatter(y=all_profit, showlegend=False),
        row=1, col=1
    )
    # Set theme, margin, and annotation in layout
    fig.update_layout(
        template="plotly_dark",
        margin=dict(r=100, t=25, b=50, l=60),
        annotations=[
            dict(
                text=f"H1 H4 atr1 kijun_sen_1 no neutral",
                showarrow=False,
                xref="paper",
                yref="paper",
                x=0,
                y=0)
        ]
    )
    fig.show()
    return all_diff, all_slope

In [ ]:
for file in os.listdir("images/1"):
    os.remove(f"images/1/{file}")
for file in os.listdir("images/0"):
    os.remove(f"images/0/{file}")
all_diff, all_slope = thread_mt5("XAUUSD")

2018-10-11 00:00:00+01:35
2022-05-07 11:07:11.936243+03:00
2018.10.11 18:00:00 new Buy order at price 1219.53
2018.10.12 20:00:00 close Buy order at price 1218.25, max diff 5.1 diff -1.28
2018.10.15 08:00:00 new Buy order at price 1221.94
2018.10.15 16:00:00 close Buy order at price 1230.68, max diff 9.84 diff 2.865
2018.10.23 11:00:00 new Buy order at price 1235.66
2018.10.23 19:00:00 close Buy order at price 1231.03, max diff 2.28 diff -5.74
2018.10.26 10:00:00 new Buy order at price 1236.43
2018.10.26 15:00:00 close Buy order at price 1233.34, max diff 0.95 diff -2.76
2018.12.04 04:00:00 new Buy order at price 1235.23
2018.12.05 02:00:00 close Buy order at price 1236.76, max diff 5.58 diff 0.755
2018.12.06 16:00:00 new Buy order at price 1243.95
2018.12.06 19:00:00 close Buy order at price 1239.62, max diff 0 diff -4.395
2018.12.07 15:00:00 new Buy order at price 1244.02
2018.12.10 15:00:00 close Buy order at price 1245.23, max diff 5.8 diff 1.245
2018.12.19 16:00:00 new Buy order a

2020.01.31 16:00:00 new Buy order at price 1582.95
2020.02.03 04:00:00 close Buy order at price 1582.85, max diff 6.47 diff -0.77
2020.02.14 13:00:00 new Buy order at price 1576.36
2020.02.14 15:00:00 close Buy order at price 1580.17, max diff 0 diff -1.28
2020.02.14 16:00:00 new Buy order at price 1582.57
2020.02.17 10:00:00 close Buy order at price 1580.34, max diff 1.33 diff -3.11
2020.02.18 02:00:00 new Buy order at price 1586.42
2020.02.20 03:00:00 close Buy order at price 1609.25, max diff 26.48 diff 19.82
2020.02.20 13:00:00 new Buy order at price 1617.55
2020.02.24 21:00:00 close Buy order at price 1653.73, max diff 69.35 diff 46.28
2020.03.05 16:00:00 new Buy order at price 1658.43
2020.03.06 17:00:00 close Buy order at price 1663.97, max diff 29.45 diff 9.545
2020.03.09 01:00:00 new Buy order at price 1698.06
2020.03.09 07:00:00 close Buy order at price 1662.55, max diff 2.84 diff -25.505
2020.03.13 18:00:00 new Sell order at price 1534.79
2020.03.16 01:00:00 close Sell order

In [ ]:
train_X, train_y = [], []
number_timestep = 50
idx = 0
for value_x, value_y in zip(X, y):
    scaler = MinMaxScaler()
    x_trans = scaler.fit_transform(value_x[['MACD','SIGNAL','HIST']][-number_timestep:])
    print(x_trans.shape)
    train_X.append(x_trans)
    train_y.append(value_y)

In [ ]:
for file in os.listdir("images/1"):
    os.remove(f"images/1/{file}")
for file in os.listdir("images/0"):
    os.remove(f"images/0/{file}")
X_raw = []
y_raw = []
X = []
y = []
y_diff = []
threads = []
for symbol in ["XAUUSD", "USDCHF", "AUDUSD", "AUDJPY", "USDJPY", "USDCAD", "BTCUSD", "USDCNH", "SpotCrude", "EURUSD"]:
    test_ = threading.Thread(target=thread_mt5, args=(symbol,), daemon=True)
    threads.append(test_)

for thread in threads:
    thread.start()
    
for thread in threads:
    thread.join()


2018-10-11 00:00:00+01:35
2022-05-06 10:49:13.349755+03:00
2018-10-11 00:00:00+01:35
2022-05-06 10:49:13.363744+03:00
2018-10-11 00:00:00+01:35
2022-05-06 10:49:13.390973+03:00
2018-10-11 00:00:00+01:35
2022-05-06 10:49:13.423867+03:00
2018-10-11 00:00:00+01:35
2022-05-06 10:49:13.443667+03:00
2018-10-11 00:00:00+01:35
2022-05-06 10:49:13.470660+03:00
2018-10-11 00:00:00+01:35
2022-05-06 10:49:13.517630+03:00
2018-10-11 00:00:00+01:35
2022-05-06 10:49:13.562704+03:00
2018-10-11 00:00:00+01:35
2022-05-06 10:49:13.590249+03:00
2018-10-11 00:00:00+01:35
2022-05-06 10:49:13.698473+03:00
2018.10.11 00:00:00 new Sell order at price 79.398
2018.10.11 00:00:00 new Sell order at price 6464.06
2018.10.11 04:00:00 new Buy order at price 6.9415700000000005
2018.10.11 12:00:00 close Sell order at price 79.801, max diff 0.075 diff -0.514
2018.10.11 12:00:00 close Buy order at price 6.90915, max diff 0 diff -0.02489
2018.10.15 08:00:00 close Sell order at price 6422.64, max diff 309.36 diff 124.435
2

2018.12.18 16:00:00 new Buy order at price 1.34844
2018.12.19 16:00:00 close Buy order at price 1.34483, max diff 0 diff -0.00667
2018.12.20 16:00:00 new Sell order at price 79.075
2019.01.10 00:00:00 new Sell order at price 0.97283
2018.12.26 20:00:00 close Sell order at price 111.362, max diff 0.916 diff -0.116
2019.01.10 16:00:00 close Sell order at price 0.98294, max diff 0.00012 diff -0.0084
2018.12.21 12:00:00 new Buy order at price 1.35415
2019.01.02 16:00:00 new Sell order at price 0.69984
2019.01.03 00:00:00 close Sell order at price 0.69364, max diff 0.00144 diff 0.0062
2018.12.31 20:00:00 new Sell order at price 109.601
2018.12.31 16:00:00 new Sell order at price 6.86843
2019.01.02 16:00:00 close Sell order at price 6.878, max diff 0.00403 diff -0.0114
2019.01.03 12:00:00 close Sell order at price 107.747, max diff 2.633 diff 1.6375
2019.01.04 08:00:00 new Sell order at price 6.87645
2019.01.04 12:00:00 close Sell order at price 6.87906, max diff 0 diff -0.00021
2019.01.04 2

2019.05.20 00:00:00 close Sell order at price 0.69139, max diff 0.0075 diff 0.00294
2019.05.06 20:00:00 close Buy order at price 6.77292, max diff 0 diff -0.04722
2019.05.31 12:00:00 new Sell order at price 55.4
2019.05.08 20:00:00 new Buy order at price 6.80758
2019.06.20 12:00:00 new Sell order at price 0.98471
2019.05.03 08:00:00 new Buy order at price 5675.0
2019.04.24 00:00:00 new Buy order at price 1.34387
2019.06.06 20:00:00 close Sell order at price 53.14, max diff 4.01 diff 2.18
2019.06.26 20:00:00 close Sell order at price 0.9779, max diff 0.01504 diff 0.00681
2019.06.20 00:00:00 new Buy order at price 1371.41
2019.04.29 20:00:00 close Buy order at price 1.34571, max diff 0.00652 diff 0.00022
2019.05.21 04:00:00 close Buy order at price 6.92824, max diff 0.13974 diff 0.11372
2019.05.06 00:00:00 new Sell order at price 76.828
2019.05.07 04:00:00 close Sell order at price 77.897, max diff 0 diff -0.919
2019.06.26 12:00:00 close Buy order at price 1409.46, max diff 64.62 diff 33

In [ ]:
print(f"total trade: {len(all_profit)} avg loss: {round(max_loss/num_loss, 1)} / {num_loss} avg profit {round(max_profit/num_profit, 1)} / {num_profit}")
print(f"total loss: {round(max_loss, 1)} total profit {max_profit} real profit {round(profit, 1)}")
print(f"all max diff avg {sum(all_max_diff) / len(all_max_diff)}")
print(f"all min diff avg {sum(all_min_diff) / len(all_min_diff)}")
print(f"ROI { round((max_profit/num_profit)/(-max_loss/num_loss), 1) }")
print(f"winrate {round(len(all_max_diff)/len(all_profit) * 100, 1)}% total trade {len(all_profit)}")

# Initialize figure with subplots
fig = make_subplots(
    rows=2, cols=1,
    column_widths=[2],
    row_heights=[1, 0.5])
#     specs=[[{"type": "scatter", "colspan": 2}, {"type": "scatter"}],
#            [            None                    , {"type": "bar"}]])
# Add locations bar chart
fig.add_trace(
    go.Bar(x=['total profit', 'total loss'],  y=[max_profit, abs(max_loss)], showlegend=False, 
           text=[f"avg profit {round(max_profit/num_profit, 2)} / {num_profit}", 
                 f"avg loss: {round(max_loss/num_loss, 2)} / {num_loss}"], textposition='auto'),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(y=all_profit, showlegend=False),
    row=1, col=1
)
# Set theme, margin, and annotation in layout
fig.update_layout(
    template="plotly_dark",
    margin=dict(r=100, t=25, b=50, l=60),
    annotations=[
        dict(
            text=f"H1 H4 atr1 kijun_sen_1 no neutral",
            showarrow=False,
            xref="paper",
            yref="paper",
            x=0,
            y=0)
    ]
)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

macd, macdsignal, macdhist = talib.MACD(df.close)
macdsignal = macdsignal.dropna()[-5:]

x = np.array([x for x in range(len(macdsignal.index))])
scaler = MinMaxScaler(feature_range=(0, 10))
y = scaler.fit_transform(macdsignal.values.reshape(-1, 1))
slope, intercept, r_value, p_value, std_err = linregress(x, y.flatten())
print("slope: %f, intercept: %f" % (slope, intercept))
print("R-squared: %f" % r_value**2)

plt.figure(figsize=(15, 5))
plt.plot(x, y, 'o', label='original data')
plt.plot(x, intercept + slope*x, 'r', label='fitted line')
plt.legend()
plt.grid()
plt.show()

In [ ]:
from datetime import datetime
import MetaTrader5 as mt5
# display data on the MetaTrader 5 package
print("MetaTrader5 package author: ",mt5.__author__)
print("MetaTrader5 package version: ",mt5.__version__)

# import the 'pandas' module for displaying data obtained in the tabular form
import pandas as pd
pd.set_option('display.max_columns', 500) # number of columns to be displayed
pd.set_option('display.width', 1500)      # max table width to display
# import pytz module for working with time zone
import pytz
 
# establish connection to MetaTrader 5 terminal
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()

# set time zone to UTC
timezone = pytz.timezone("Etc/UTC")
# create 'datetime' objects in UTC time zone to avoid the implementation of a local time zone offset
utc_to = datetime.utcnow() + timedelta(days = 0, hours = 9, minutes=0)
utc_from = utc_to - timedelta(days = 0, hours = 10, minutes=0)
# request AUDUSD ticks within 11.01.2020 - 11.01.2020
ticks = mt5.copy_ticks_range("XAUUSD", utc_from, utc_to, mt5.COPY_TICKS_ALL)
print("Ticks received:",len(ticks))
 
# shut down connection to the MetaTrader 5 terminal
# mt5.shutdown()
#  
# display data on each tick on a new line
print("Display obtained ticks 'as is'")
count = 0
for tick in ticks:
    count+=1
    print(tick)
    if count >= 10:
        break

# create DataFrame out of the obtained data
ticks_frame = pd.DataFrame(ticks)
# convert time in seconds into the datetime format
ticks_frame['time']=pd.to_datetime(ticks_frame['time'], unit='s')
ticks_frame['price'] = (ticks_frame['bid'] + ticks_frame['ask']) / 2
# display data
print("\nDisplay dataframe with ticks")
print(ticks_frame.tail(10)) 

In [ ]:
# request AUDUSD ticks within 11.01.2020 - 11.01.2020
ticks = mt5.copy_ticks_range("XAUUSD", utc_from, utc_to, mt5.COPY_TICKS_ALL)
print("Ticks received:",len(ticks))
# create DataFrame out of the obtained data
ticks_frame = pd.DataFrame(ticks)
# convert time in seconds into the datetime format
ticks_frame['time']=pd.to_datetime(ticks_frame['time'], unit='s')
ticks_frame['price'] = (ticks_frame['bid'] + ticks_frame['ask']) / 2
ticks_frame = ticks_frame[['time','bid','ask','price']]
# print(ticks_frame.tail())
ticks_frame.set_index('time').loc[:,'bid':'ask']
datetime_index = pd.DatetimeIndex(ticks_frame.time)
df2=ticks_frame.set_index(datetime_index)
OHLCData = df2.resample('5min').agg({'bid':'max','ask':'max','price':'ohlc'})
OHLCData

In [ ]:
import datetime as dt
import matplotlib.pyplot as plt
import pandas_ta as ta

rates_frame = mt5.copy_rates_from_pos('BTCUSD', mt5.TIMEFRAME_M30, 0, 100)
#     print(rates_frame)
# create DataFrame out of the obtained data
df = pd.DataFrame(rates_frame, columns=['time', 'open', 'high', 'low', 'close'])
df['Date'] = pd.to_datetime(df['time'], unit='s')
df = df.drop(['time'], axis=1)
df = df.reindex(columns=['Date', 'open', 'high', 'low', 'close'])

# convert to float to avoid sai so.
df.open = df.open.astype(float)
df.high = df.high.astype(float)
df.low = df.low.astype(float)
ticker = df.close.astype(float)

# Get the 26-day EMA of the closing price
k = df['close'].ewm(span=12, adjust=False, min_periods=12).mean()
# Get the 12-day EMA of the closing price
d = df['close'].ewm(span=26, adjust=False, min_periods=26).mean()
# Subtract the 26-day EMA from the 12-Day EMA to get the MACD
macd = k - d
# Get the 9-Day EMA of the MACD for the Trigger line
macd_s = macd.ewm(span=9, adjust=False, min_periods=9).mean()
# Calculate the difference between the MACD - Trigger for the Convergence/Divergence value
macd_h = macd - macd_s
# Add all of our new values for the MACD to the dataframe
# df['macd'] = df.index.map(macd)
# df['macd_h'] = df.index.map(macd_h)
# df['macd_s'] = df.index.map(macd_s)

macd.plot(label='AAPL MACD', color='g')
ax = macd_s.plot(label='Signal Line', color='r')
ticker.plot(ax=ax, secondary_y=True, label='AAPL')

ax.set_ylabel('MACD')
ax.right_ax.set_ylabel('Price $')
ax.set_xlabel('Date')
lines = ax.get_lines() + ax.right_ax.get_lines()
ax.legend(lines, [l.get_label() for l in lines], loc='upper left')
plt.show()

# training

In [ ]:
import pandas
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from keras.callbacks import EarlyStopping, ModelCheckpoint
from livelossplot import PlotLossesKeras
from sklearn.metrics import plot_confusion_matrix, classification_report

In [ ]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

In [ ]:
from sklearn.model_selection import train_test_split
X_train_tmp = []
y_train_tmp = []
for x_tmp, y_tmp in zip(X, y):
#     if y_tmp == 1:
    X_train_tmp.append(np.array(x_tmp).flatten())
    y_train_tmp.append(y_tmp)
    
X_train = np.array(X_train_tmp)
y_train = np.array(y_train_tmp)
# X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size=0.2, random_state=42)
# scaler = MinMaxScaler()
# X_train = scaler.fit_transform(X_new)
# y_train = y_new
# X_test = scaler.transform(X_test)

# n_in = len(X_train)
# sequence = X_train.reshape((n_in, 100, 6))

In [ ]:
X_train.shape

In [ ]:
import plotly.express as px
df = pd.DataFrame(y_train, columns=["price"])
fig = px.histogram(df, x="price")
fig.show()

In [ ]:
plt.scatter(X_train[:, 0], X_train[:, 1], cmap='viridis')

In [ ]:
import joblib
scaler_filename = "scaler.save"
joblib.dump(scaler, scaler_filename) 

# Auto encoder

In [ ]:
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
# from keras.utils import plot_model

In [ ]:
# define model
model = Sequential()
model.add(LSTM(128, activation='relu', input_shape=(100,6)))
model.add(RepeatVector(100))
model.add(LSTM(128, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(6)))
model.compile(optimizer='adam', loss='mse')
# fit model
# ADAM = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False, clipnorm=1.)
# model.compile(optimizer=ADAM, loss='mse')
model.fit(sequence, sequence, epochs=20, verbose=1)

In [ ]:
# plot_model(model, show_shapes=True, to_file='reconstruct_lstm_autoencoder.png')
# demonstrate recreation
yhat = model.predict(sequence, verbose=0)
print(yhat[0,:,0])

In [ ]:
print(sequence[0,:,0])

In [ ]:
def make_model(input_shape):
    input_layer = keras.layers.Input(input_shape)

    conv1 = keras.layers.Conv1D(filters=8, kernel_size=3, padding="same")(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.Activation("relu")(conv1)

    conv2 = keras.layers.Conv1D(filters=8, kernel_size=3, padding="same")(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.Activation("relu")(conv2)

    conv3 = keras.layers.Conv1D(filters=8, kernel_size=3, padding="same")(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.Activation("relu")(conv3)

    gap = keras.layers.GlobalAveragePooling1D()(conv3)

    output_layer = keras.layers.Dense(1, activation="sigmoid")(gap)

    return keras.models.Model(inputs=input_layer, outputs=output_layer)


model = make_model(input_shape=X_train.shape[1:])

In [ ]:
epochs = 1000
batch_size = 16

callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model.h5", save_best_only=True, monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001
    ),
#     keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1),
    PlotLossesKeras()
]
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"],
)

history = model.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=(X_test, y_test),
    verbose=1,
)

In [ ]:
model = keras.models.load_model("best_model.h5")

test_loss, test_acc = model.evaluate(X_test, y_test)

print("Test accuracy", test_acc)
print("Test loss", test_loss)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans, KMeans
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics.pairwise import pairwise_distances_argmin
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
from sklearn import metrics
from scipy.spatial.distance import cdist
from sklearn.cluster import SpectralClustering

In [ ]:
distortions = []
inertias = []
mapping1 = {}
mapping2 = {}
K = range(1, 10)
 
for k in K:
    # Building and fitting the model
    kmeanModel = KMeans(n_clusters=k)
    kmeanModel.fit(X_train)
 
    distortions.append(sum(np.min(cdist(X_train, kmeanModel.cluster_centers_, 'euclidean'), axis=1)) /X_train.shape[0])
    inertias.append(kmeanModel.inertia_)
 
    mapping1[k] = sum(np.min(cdist(X_train, kmeanModel.cluster_centers_, 'euclidean'), axis=1)) / X_train.shape[0]
    mapping2[k] = kmeanModel.inertia_
plt.plot(K, distortions, 'bx-')
plt.xlabel('Values of K')
plt.ylabel('Distortion')
plt.title('The Elbow Method using Distortion')
plt.show()

In [ ]:
x_plot = X_train[:, 0]
y_plot = X_train[:, 1]
plt.scatter(x_plot, y_plot, s=y_train, cmap='viridis')

In [ ]:
x_train_pca.shape

In [ ]:
pca = PCA(n_components=2)
x_train_pca = pca.fit_transform(X_train)
# x_test_pca = pca.transform(X_test)

x_plot = x_train_pca[:, 0]
y_plot = x_train_pca[:, 1]

t0 = time.time()
n_clusters = 3
k_means_model = KMeans(n_clusters=n_clusters, random_state=0, max_iter=1000).fit(X_train)
sprectal_clustering_model = SpectralClustering(n_clusters=n_clusters, affinity='nearest_neighbors', assign_labels='kmeans')
# logistic = LogisticRegression(random_state=0, max_iter=10000).fit(X_train, y_train)
# liner_model = LinearRegression().fit(X_train, y_train)
t_batch = time.time() - t0

k_means_model.labels_

def set_color(value):
    return "r" if value < 0 else "g"

def set_sizes(value):
    return abs(value) * 10

y_pred = k_means_model.predict(X_train)
# yhat_probs = logistic.predict_proba(X_train)[:, 1]
# reduce to 1d array
# convert = lambda x: 1 if x > 0.9 else 0
# y_pred = list(map(convert, yhat_probs ))

sizes = list(map(set_sizes, y_train))
colors = list(map(set_color, y_train))
plt.scatter(x_plot, y_plot, c=colors, s=sizes, cmap='viridis')

# centers = k_means_model.cluster_centers_
# plt.scatter(centers[:, 0], centers[:, 1], c='red', s=200, alpha=0.5);

# colors = list(map(set_color, y_pred))

# x_plot = x_train_pca[:, 0]
# y_plot = x_train_pca[:, 1]
# plt.scatter(x_plot, y_plot, c=colors,s=sizes, alpha=1)
plt.colorbar();  # show color scale

# clusters_profit = [0]*n_clusters
for i, (label_train, predict_cluster) in enumerate(zip(y_train, y_pred)):
#     center_point = centers[predict_cluster]
#     distance = np.linalg.norm(center_point-(x_plot[i], y_plot[i]))
#     if distance < 3:
    plt.annotate(f"{round(label_train, 1)}", (x_plot[i], y_plot[i]))
#     plt.scatter(x_plot[i], y_plot[i], c=predict_cluster, s=label_train, cmap='viridis')

In [ ]:
px.histogram(y_train)

In [ ]:
y_pred

In [ ]:
clusters_profit

In [ ]:
y_pred = k_means_model.predict(X_test)
colors = list(map(set_color, y_pred))
sizes = list(map(set_sizes, y_pred))
x_plot = x_test_pca[:, 0]
y_plot = x_test_pca[:, 1]
plt.scatter(x_plot, y_plot, c=colors, s=sizes)
plt.title("Incorrect Number of Blobs")
for i, txt in enumerate(y_test):
    plt.annotate(round(txt, 1), (x_plot[i], y_plot[i]))

In [ ]:
# model = keras.models.load_model("best_model.h5")

# test_loss, test_acc = model.evaluate(X_test, y_test)

# print("Test accuracy", test_acc)
# print("Test loss", test_loss)

In [ ]:
y_pred

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
#Confution Matrix and Classification Report
#Y_pred = model.predict(X_test)
#y_pred = np.argmax(Y_pred, axis=0)

# predict probabilities for test se
y_pred = logistic.predict(X_train)
# reduce to 1d array
# convert = lambda x: 1 if x > 0.5 else 0
# y_pred = list(map(convert, y_pred ))

print('Confusion Matrix')
print(confusion_matrix(y_train, y_pred))
print('Classification Report')
target_names = ['0', '1']
print(classification_report(y_train, y_pred, target_names=target_names))

In [ ]:
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot
from sklearn.metrics import roc_curve
# ROC AUC
yhat_probs = logistic.predict_proba(X_train)[:, 1]
auc = roc_auc_score(y_train, yhat_probs)
print('ROC AUC: %f' % auc)

In [ ]:
# predict probabilities
lr_probs = logistic.predict_proba(X_train)[:, 1]
# keep probabilities for the positive outcome only
ns_probs = [0 for _ in range(len(X_train))]
ns_auc = roc_auc_score(y_train, ns_probs)
lr_auc = roc_auc_score(y_train, lr_probs)
# summarize scores
print('No Skill: ROC AUC=%.3f' % (ns_auc))
print('Logistic: ROC AUC=%.3f' % (lr_auc))
# calculate roc curves
ns_fpr, ns_tpr, ns_thresholds  = roc_curve(y_train, ns_probs)
lr_fpr, lr_tpr, lr_thresholds  = roc_curve(y_train, lr_probs)
# plot the roc curve for the model
pyplot.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
pyplot.plot(lr_fpr, lr_tpr, linestyle='--', label='Logistic')
# axis labels
pyplot.xlabel('False Positive Rate')
pyplot.ylabel('True Positive Rate')
# show the legend
pyplot.legend()
# show the plot
pyplot.show()

In [ ]:
lr_thresholds

In [ ]:
lr_tpr

In [ ]:
lr_fpr

In [ ]:
def get_tick_data(utc_from, utc_to, timeframe, symbol):
    mapping_timeframe = {
        mt5.TIMEFRAME_H1: "60min",
        mt5.TIMEFRAME_M15: "15min",
        mt5.TIMEFRAME_M5: "5min",
        mt5.TIMEFRAME_M30: "30min",
        mt5.TIMEFRAME_D1: "D1",
        mt5.TIMEFRAME_H4: "H4",
    }
    map_timeframe = mapping_timeframe.get(timeframe)
    # request AUDUSD ticks within 11.01.2020 - 11.01.2020
    ticks = mt5.copy_ticks_range(symbol, utc_from, utc_to, mt5.COPY_TICKS_ALL)
#     print("Ticks received:",len(ticks))
    # create DataFrame out of the obtained data
    ticks_frame = pd.DataFrame(ticks)
    # convert time in seconds into the datetime format
    ticks_frame['time']=pd.to_datetime(ticks_frame['time'], unit='s')
    ticks_frame['price'] = (ticks_frame['bid'] + ticks_frame['ask']) / 2
    ticks_frame = ticks_frame[['time','bid','ask','price']]
    # print(ticks_frame.tail())
    ticks_frame.set_index('time').loc[:,'bid':'ask']
    datetime_index = pd.DatetimeIndex(ticks_frame.time)
    ticks_frame=ticks_frame.set_index(datetime_index)
    OHLCData = ticks_frame.resample(map_timeframe).agg({'bid':'max','ask':'max','price':'ohlc'})
    df = pd.DataFrame(OHLCData.price, columns=['open', 'high', 'low', 'close'])
    df['bid'] = OHLCData.bid['bid']
    df['ask'] = OHLCData.ask['ask']
    df['Date'] = OHLCData.index
    df = df.reindex(columns=['Date', 'open', 'high', 'low', 'close', 'bid', 'ask'])
    df = df.reset_index()
    df = df.drop(['time'], axis=1)
    #df['EMA_50'] = df.close.ewm(span=50, adjust=False).mean()
    df['EMA12'] = df.close.ewm(span=12, adjust=False).mean()
    df['EMA26'] = df.close.ewm(span=26, adjust=False).mean()
    df['MACD'] = df['EMA26'] - df['EMA12']
    df['MACDSignalLine'] = df['MACD'].ewm(span=9, adjust=False).mean()
    df['HIST'] = df['MACD'] - df['MACDSignalLine']
    atr_multiplier = 3.0
    atr_period = 10
    supertrend = Supertrend(df, atr_period, atr_multiplier)
    df = df.join(supertrend)
    lookback = 14
    conditions = [
        (df['Supertrend10'] == True) & (df['HIST'] > 0) & (df['HIST'] > df['HIST'].shift(2)),
        (df['Supertrend10'] == False) & (df['HIST'] < 0) & (df['HIST'] < df['HIST'].shift(2))
    ]
    values = ['Buy', 'Sell']
    df['Trend'] = np.select(conditions, values)
#     print(df.tail(10))
    return df.Date.iat[-1], df.Trend.iat[-1], df.bid.iat[-1], df.ask.iat[-1], 

def Supertrend(df, atr_period, multiplier):
    
    high = df['high']
    low = df['low']
    close = df['close']
    
    # calculate ATR
    price_diffs = [high - low, 
                   high - close.shift(), 
                   close.shift() - low]
    true_range = pd.concat(price_diffs, axis=1)
    true_range = true_range.abs().max(axis=1)
    # default ATR calculation in supertrend indicator
    atr = true_range.ewm(alpha=1/atr_period,min_periods=atr_period).mean() 
    # df['atr'] = df['tr'].rolling(atr_period).mean()
    
    # HL2 is simply the average of high and low prices
    hl2 = (high + low) / 2
    # upperband and lowerband calculation
    # notice that final bands are set to be equal to the respective bands
    final_upperband = upperband = hl2 + (multiplier * atr)
    final_lowerband = lowerband = hl2 - (multiplier * atr)
    
    # initialize Supertrend column to True
    supertrend = [True] * len(df)
    
    for i in range(1, len(df.index)):
        curr, prev = i, i-1
        
        # if current close price crosses above upperband
        if close[curr] > final_upperband[prev]:
            supertrend[curr] = True
        # if current close price crosses below lowerband
        elif close[curr] < final_lowerband[prev]:
            supertrend[curr] = False
        # else, the trend continues
        else:
            supertrend[curr] = supertrend[prev]
            
            # adjustment to the final bands
            if supertrend[curr] == True and final_lowerband[curr] < final_lowerband[prev]:
                final_lowerband[curr] = final_lowerband[prev]
            if supertrend[curr] == False and final_upperband[curr] > final_upperband[prev]:
                final_upperband[curr] = final_upperband[prev]

        # to remove bands according to the trend direction
        if supertrend[curr] == True:
            final_upperband[curr] = np.nan
        else:
            final_lowerband[curr] = np.nan
    
    return pd.DataFrame({
        f'Supertrend{atr_period}': supertrend,
        f'FinalLowerband{atr_period}': final_lowerband,
        f'FinalUpperband{atr_period}': final_upperband
    }, index=df.index)

In [ ]:
import numpy as np
from scipy.signal import argrelextrema
import matplotlib.pyplot as plt
from scipy.signal import find_peaks, find_peaks_cwt

In [ ]:
current_time = datetime.now().hour % 4
rates_frame = mt5.copy_rates_from_pos("XAUUSD", mt5.TIMEFRAME_H4, 0, 100)
df = pd.DataFrame(rates_frame, columns=['time', 'open', 'high', 'low', 'close'])
df['Date'] = pd.to_datetime(df['time'], unit='s')
df = df.drop(['time'], axis=1)
df = df.reindex(columns=['Date', 'open', 'high', 'low', 'close'])
df['macd'], df['signal'], df['histogram'] = talib.MACD(df.close)
df['rsi'] = talib.RSI(df.close)
df['atr'] = talib.ATR(df.high, df.low, df.close)
df = df.dropna()
df.tail()


last_point = len(df.signal.values)
x = df.signal.values
close = df.close.values
macd = df.macd.values
scaler = MinMaxScaler(feature_range=(np.min(x), np.max(x)))
close = scaler.fit_transform(close.reshape(-1, 1)).flatten()

x1 = np.array([tmp for tmp in x])
maxima = argrelextrema(x1, np.greater)

x2 = np.array([tmp for tmp in x])
minima = argrelextrema(x2, np.less)

plt.figure(figsize=(15, 5))
plt.plot([x for x in range(len(x))], x)

plt.scatter(maxima, [x[min_val] for min_val in maxima])
plt.scatter(minima, [x[min_val] for min_val in minima])
plt.plot([x for x in range(len(close))], x)
plt.plot([x for x in range(len(close))], macd)
plt.show()

np.intersect1d(df['macd'].values, df['signal'].values)

In [ ]:
maxima[0][-1]

In [ ]:
len(x)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
checkpoint_filepath = './checkpoint'

In [ ]:
train_size = int(len(df) * 0.9)
test_size = len(df) - train_size
train, test = df.iloc[0:train_size], df.iloc[train_size:len(df)]
test = test.reset_index(drop=True)
train.shape

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from joblib import dump, load
features = ['histogram', 'macd', 'signal']
scaler = MinMaxScaler()
# scaler = load('scaler.save') 
train_scaler = scaler.fit_transform(train[features])
test_scaler = scaler.transform(test[features])

In [ ]:
train_scaler.shape

In [ ]:
# split a univariate sequence into samples
def split_sequence(raw_frame, sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix >= len(sequence):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], end_ix
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [ ]:
def create_dataset(X, time_steps=1):
    X1, y1 = [], []
    for i in range(len(X) - time_steps):
        t = X[i:(i + time_steps)]
        X1.append(t)
    return np.array(X1)
TIME_STEPS = 50
X_train, y_train = split_sequence(
    train,
    train_scaler,
    TIME_STEPS
)
X_test, y_test = split_sequence(
    test,
    test_scaler,
    TIME_STEPS
)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
# X_train = np.array(train_X)
# train_y = np.array(train_y)

In [ ]:
timesteps, input_dim = X_train.shape[1], X_train.shape[2]
latent_dim = 64
inputs = tf.keras.Input(shape=(timesteps, input_dim))
encoded = tf.keras.layers.LSTM(64, return_sequences=True)(inputs)
encoded = tf.keras.layers.Dropout(rate=0.2)(encoded)
encoded = tf.keras.layers.LSTM(latent_dim, return_sequences=True)(encoded)
encoded = tf.keras.layers.LSTM(latent_dim)(encoded)
decoded = tf.keras.layers.RepeatVector(timesteps)(encoded)
decoded = tf.keras.layers.LSTM(latent_dim, return_sequences=True)(decoded)
# decoded = tf.keras.layers.LSTM(64, return_sequences=True)(encoded)
decoded = tf.keras.layers.Dropout(rate=0.2)(decoded)
decoded = tf.keras.layers.LSTM(input_dim, return_sequences=True)(decoded)

sequence_autoencoder = tf.keras.Model(inputs, decoded)
encoder = tf.keras.Model(inputs, encoded)
sequence_autoencoder.compile(loss='mse', optimizer='adam')
# sequence_autoencoder.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
sequence_autoencoder.summary()

In [ ]:
import tensorflow as tf

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=100)

history = sequence_autoencoder.fit(
    X_train, X_train,
    epochs=2000,
    batch_size=32,
    validation_split=0.3,
    shuffle=False,
    callbacks=[callback, model_checkpoint_callback]
)

In [ ]:
sequence_autoencoder.load_weights(checkpoint_filepath)

In [ ]:
# from sklearn.metrics import classification_report

# y_pred = sequence_autoencoder.predict(X_train, batch_size=64, verbose=1)
# y_pred_bool = np.argmax(y_pred, axis=1)

# print(classification_report(train_y, y_pred_bool))

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.show()

In [ ]:
# pred = sequence_autoencoder.predict(X_test, verbose=0)

In [ ]:
embedding = encoder.predict(X_train, verbose=0)
embedding_test = encoder.predict(X_test, verbose=0)

In [ ]:
# tmp_scaler = MinMaxScaler()
# embedding = tmp_scaler.fit_transform(embedding)
# embedding_test = tmp_scaler.transform(embedding_test)

In [ ]:
# from joblib import dump, load
# dump(scaler, 'scaler.save') 

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from mpl_finance import candlestick_ohlc
import matplotlib.dates as mpl_dates
import matplotlib.dates as md
%matplotlib notebook

In [ ]:
width = .4
width2 = .05
#define colors to use
col1 = 'green'
col2 = 'red'

In [ ]:
# np.save('embedding.npy', embedding)

In [ ]:
fig, subfig = plt.subplots(4, 1)
plot_price = False
plt.ion()
fig.show()
fig.canvas.draw()
for plot_point in range(len(y_test) - 1):
    sample = embedding_test[plot_point]
    min_dist = 100
    curr = []
    all_dist = []

    similar_raw = cosine_similarity([sample], embedding)[0][1:]
    top_maximize = similar_raw.argsort()[-4:][::-1]
    similar_values = []
    for item in top_maximize:
        similar_values.append(similar_raw[item])
    print(similar_values, top_maximize)
    
    ax1 = subfig[0]
    ax1.clear()
    ax1.title.set_text(f'{test.Date.values[y_test[plot_point]]}%')
    fig.suptitle('Horizontally stacked subplots')
    ohlc = test.iloc[(plot_point):(plot_point+TIME_STEPS+TIME_STEPS)]
    up = ohlc[ohlc.close>=ohlc.open]
    down = ohlc[ohlc.close<ohlc.open]

    if plot_price:
        ax1.bar(up.index,up.close-up.open,width,bottom=up.open,color=col1)
        ax1.bar(up.index,up.high-up.close,width2,bottom=up.close,color=col1)
        ax1.bar(up.index,up.low-up.open,width2,bottom=up.open,color=col1)
        ax1.bar(down.index,down.close-down.open,width,bottom=down.open,color=col2)
        ax1.bar(down.index,down.high-down.open,width2,bottom=down.open,color=col2)
        ax1.bar(down.index,down.low-down.close,width2,bottom=down.close,color=col2)
        ax1.scatter([test.index[y_test[plot_point]]], [test.close.values[y_test[plot_point]]])
    else:
        ax1.plot(ohlc.index, ohlc.macd.values)
        ax1.plot(ohlc.index, ohlc.histogram.values)
        ax1.plot(ohlc.index, ohlc.signal.values)
        ax1.scatter([test.index[y_test[plot_point]]], [test.macd.values[y_test[plot_point]]])

    new_ploted = False
    for ax, plot_point, similar_value in zip(subfig[1:], top_maximize, similar_values):
        ax.clear()
        ax.title.set_text(f'{train.Date.values[y_train[plot_point]]} Similar {similar_value * 100}%')

        ohlc = train.iloc[(plot_point):(plot_point+TIME_STEPS+TIME_STEPS)]
        up = ohlc[ohlc.close>=ohlc.open]
        down = ohlc[ohlc.close<ohlc.open]
        if plot_price:
            ax.bar(up.index,up.close-up.open,width,bottom=up.open,color=col1)
            ax.bar(up.index,up.high-up.close,width2,bottom=up.close,color=col1)
            ax.bar(up.index,up.low-up.open,width2,bottom=up.open,color=col1)
            ax.bar(down.index,down.close-down.open,width,bottom=down.open,color=col2)
            ax.bar(down.index,down.high-down.open,width2,bottom=down.open,color=col2)
            ax.bar(down.index,down.low-down.close,width2,bottom=down.close,color=col2)
            ax.scatter([train.index[y_train[plot_point]]], [train.close.values[y_train[plot_point]]])
        else:
            ax.plot(ohlc.index, ohlc.macd.values)
            ax.plot(ohlc.index, ohlc.histogram.values)
            ax.plot(ohlc.index, ohlc.signal.values)
            ax.scatter([train.index[y_train[plot_point]]], [train.macd.values[y_train[plot_point]]])
        new_ploted = True
    fig.canvas.draw()